# Scraper for FlavorDB

In [1]:
import requests
import json
import pandas as pd
import csv

Captures only entities with full data, but it seems incomplete also have adequate molecule data

In [4]:
json_res = requests.get('https://cosylab.iiitd.edu.in/flavordb/entities?entity=%20&category=')
text_res = json.loads(json_res.text)
res = json.loads(text_res)

Store all entities in `data.csv`

In [63]:
data = pd.DataFrame.from_records(res)
data.to_csv('data.csv', index=False)

with open('data.csv') as file:
  csv_reader = csv.DictReader(file)
  data = [row for row in csv_reader]

Read in all entity information into `data_chemicals.csv` (approx. 35 minutes on M1)

In [5]:
# molecules stored in indices from 0 to 978
id = 0
with open('data_chemicals.csv', 'a') as csvfile:
  writer = csv.writer(csvfile)
  # csv header
  writer.writerow(['id', 'common_name', 'category', 'chemicals'])
  while id <= 978:
    try:
      entity_res1 = requests.get(f'https://cosylab.iiitd.edu.in/flavordb/entities_json?id={id}')
      entity_res = json.loads(entity_res1.text)
      category = entity_res['category']
      common_name = entity_res['entity_alias_readable']
      # !! May want to grab functional groups here from entity_res['molecules'][i]['functional_groups']
      molecules = list(map(lambda obj: obj['common_name'], entity_res['molecules']))
      writer.writerow([id, common_name, category, str(molecules)])
    except:
      print(id)
    finally:
      id += 1

406
407
414
420


In [11]:
import ast

with open('data_chemicals.csv', 'r') as file:
  csv_reader = csv.DictReader(file)
  # convert stringified list back into python list
  data = [ast.literal_eval(row['chemicals']) for row in csv_reader]

# getting distinct chemicals
chemical_set = set()
for lst in data:
  chemical_set.update(lst)

print(len(chemical_set))


1781


TODO:
- remove entities with duplicate chemical compounds
- one-hot encode according to chemical_set